# Extracting the maximum intensity curve from the image

As suspected from \autoref{fig:lines} the extraction of the maximum intensity arc is not as trivial as in the smooth-gradient MHD case. This will not change when introducing the \ac{SXI} \ac{FOV} and the satellite's \ac{POV}, but it will improve since the instrument never points directly to the cusps, while image processing techniques can also be applied to extract continuous arcs.

## Constructing the images

To construct the synthetic images of the \ac{SXI} instrument, we use the \href{https://3dview.irap.omp.eu/}{3DView}  open-source tool developed by CDPP \cite{3DView}. Through this tool we can simulate the orbit and attitude of the satellite for a specified interval, as well as the orientation and characteristics of the imaging system in relation to the satellite. We can directly import the emissivity cubes of the simulations, introduce the satellite and the \ac{FOV} of the imager, and compute the integrated flux over each pixel's \ac{LOS}. We therefore get the integrated images from each \ac{POV} of the orbit, according to the \ac{FOV} and resolution of the imager, as shown in \autoref{3DView}. No simulated noise has been added to account for the detection system of the instrument or the background contribution. A typical synthetic image of the magnetopause, generated through this method, is shown in \autoref{fig:typical_image}.

```{figure} ./images/3DView.png
:name: 3DView
3D View CDPP software
```


Along with the images, we can export XML files that contain information about the position and orientation of the satellite, for each time-step. This will be particularly useful in \autoref{section: tangent_hypothesis}.

(max_int_arc)=
## Image processing

Looking at the images, we notice that two arcs seem to form, the outer one being less distinct. We will refer to that as the second maxima arc, as this second peak is observed consistently when tracing the intensity evolution over the azimuth angles for each row as seen in \autoref{fig:intensity_azimuth}. This shall not be confused with the second emissivity maximum described in the previous section, where we were referring to the 3D structures of the emissivity.


If we attempt to trace the maximum pixel per row-elevation angle, we will notice prominent jumps from the maximum intensity arc to the second maximum, especially towards the edges where the intensity of the inner arc drops relative to the outer one, as in \autoref{fig:jumps}. Attempts to discard the jumps directly proved unsuccessful since the limit in the azimuth step between rows simply slowly moved the curve to the outer arc.

Another approach to discard the jumps and limit ourselves to the arcs of interest would be to fit a polynomial to the noisy line. When trying to fit a 2nd polynomial, we have 3 degrees of freedom and therefore a 3 dimensional parameter space. 
$$x = A(y-B)^2 + C$$
This is relatively fast but cannot take into account the asymmetry of the true arc (\autoref{fig:poly2nd}). If we introduce a $y^3$ term, this anisotropy can theoretically be fitted, although in practice the 3rd order term blows up, and the resulting curve rarely converges (even with very small initial coefficients given, as in \autoref{fig:poly3rd} and \autoref{fig:poly3rd-fail}).

\begin{figure}[bth]
    \myfloatalign

    \subfloat[Second order polynomial.]
    {
    \label{fig:poly2nd}
    \includegraphics[width=0.3\textwidth]{gfx/max_arc/poly2nd.png}} 
    \quad
    \subfloat[Third order polynomial.]
    {
    \label{fig:poly3rd}
    \includegraphics[width=0.3\textwidth]{gfx/max_arc/poly3rd2.png}} 
    \quad
    \subfloat[Third order polynomial.]
    {
    \label{fig:poly3rd-fail}
    \includegraphics[width=0.3\textwidth]{gfx/max_arc/poly3rd.png}} 
    \quad

    \caption{Polynomials fitted to noisy maximum intensity arc.}  
    \label{fig:poly}
\end{figure}


Shortly, it is evident that not only is it a slow solution and without a measure of the error - but it is also difficult to implement while ensuring convergence and not overfitting. Thus, we move back to the maximum extraction for each line approach, where we need to discard the noise that causes the jumps. 

There are several image processing tools and libraries that can aide us in smoothing the image or increasing the contrast to reduce the noise and make some features more distinct. Here we increase the contrast of the images using the \href{https://scikit-image.org/docs/stable/api/skimage.html}{skimage} library and apply a 1 sigma Gaussian filter to smooth the noise. We then find the maximum azimuth index for each elevation and discard the ones that have a jump larger than 2 indexes. In the pictures approaching the magnetopause random arcs are being detected, so we discard the filtered array that are smaller than 20 points (the rest have been discarded due to jumps), as well as the ones that have an average intensity smaller than 0.2 normalized, since low intensities are expected inside the magnetopause. The result is shown in \autoref{fig:contrast}.

\begin{figure}[bth]
    \myfloatalign

    \subfloat[Increased contrast and Gaussian filter.]
    {
    \label{fig:contrast}
    \includegraphics[width=0.45\textwidth]{gfx/max_arc/contrast_gaussian.png}} 
    \quad
    \subfloat[Edge detection filter.]
    {
    \label{fig:edge_sobel}
    \includegraphics[width=0.45\textwidth]{gfx/max_arc/sobel.png}} 
    \quad

    \caption{Arc detection after different filters and imaging processing techniques have been applied.}  
    \label{fig:image_processing}
\end{figure}

We can apply a similar procedure to derive diagnostics about the maximum gradient arc as well. We do this by passing the images through an edge detection filter - \textit{skimage.filters.sobel} - which by definition extracts the maximum gradients in an image. A typical image after this filter is shown in \autoref{fig:edge_sobel}. 
